In [15]:
import requests
import pandas as pd
import numpy as np
from pprint import pprint

url = 'https://fantasy.premierleague.com/api/bootstrap-static/'


In [16]:
response = requests.get(url)
json = response.json()

In [17]:
elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])
events_df = pd.DataFrame(json["events"])


In [18]:
elements_df.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'influence_rank', 'influence_rank_type', 'creativity_rank',
       'creativity_rank_type', 'threat_rank'

In [19]:
players_df = elements_df[['second_name','first_name','team','element_type','selected_by_percent',
       'minutes','value_season','total_points','bps', 
       'influence', 'creativity', 'threat','goals_scored','assists','clean_sheets', 'goals_conceded', 
       'own_goals','penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards','saves', 'bonus']]

In [20]:
players_df['position'] = players_df.element_type.map(elements_types_df.set_index('id').singular_name)

C:\Users\steve\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [21]:
players_df.head()

,second_name,first_name,team,element_type,selected_by_percent,minutes,value_season,total_points,bps,influence,...,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,position
0,Leno,Bernd,1,1,1.8,270,0.8,4,48,79.0,...,0,9,0,0,0,0,0,9,0,Goalkeeper
1,Rúnarsson,Rúnar Alex,1,1,1.5,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,Goalkeeper
2,Borges Da Silva,Willian,1,3,0.1,0,0.0,0,0,0.0,...,0,0,0,0,0,0,0,0,0,Midfielder
3,Aubameyang,Pierre-Emerick,1,4,1.3,87,0.2,2,5,5.0,...,0,4,0,0,0,0,0,0,0,Forward
4,Soares,Cédric,1,2,0.2,180,0.0,0,16,21.4,...,0,7,0,0,0,1,0,0,0,Defender


In [22]:
players_df['team'] = players_df.team.map(teams_df.set_index('id').name)
players_df['value_season'] = players_df.value_season.astype(float)

C:\Users\steve\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [23]:
players_df = players_df.sort_values('value_season', ascending = False)


In [24]:
players_df = players_df.loc[players_df["minutes"] >= 10]
try:
    players_df["Avg Playtime"] = players_df['minutes'] / 38
except:
    0
try:
    players_df["PPG"] = players_df['total_points'] / 38
except:
    0
try:
    players_df["PPGP"] = players_df["total_points"] / (players_df["minutes"]/90)
except:
    0
players_df.corr()

,element_type,minutes,value_season,total_points,bps,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,Avg Playtime,PPG,PPGP
element_type,1.000000,-0.323987,-0.186778,-0.036223,-0.283367,0.296699,0.144905,-0.123557,-0.173736,-0.067423,-0.117731,0.092655,0.071043,-0.001477,-0.504492,0.069504,-0.323987,-0.036223,0.227383
minutes,-0.323987,1.000000,0.625589,0.588628,0.758497,0.235693,0.185739,0.428869,0.506090,0.070681,0.054378,0.053185,0.264559,-0.021631,0.233016,0.298775,1.000000,0.588628,-0.188950
value_season,-0.186778,0.625589,1.000000,0.931198,0.860139,0.465572,0.419445,0.657997,-0.033178,-0.021911,0.036411,0.197240,0.049768,0.009177,0.193831,0.690726,0.625589,0.931198,0.415721
total_points,-0.036223,0.588628,0.931198,1.000000,0.868162,0.661292,0.514183,0.638013,-0.056706,-0.029062,0.013679,0.278708,0.065751,0.003961,0.138186,0.793069,0.588628,1.000000,0.481179
bps,-0.283367,0.758497,0.860139,0.868162,1.000000,0.488867,0.442293,0.509078,0.200692,0.005996,0.048744,0.274722,0.073170,-0.026354,0.241457,0.661146,0.758497,0.868162,0.258879
goals_scored,0.296699,0.235693,0.465572,0.661292,0.488867,1.000000,0.253284,0.171580,0.027777,0.032709,-0.023376,0.348380,0.116247,0.013211,-0.100167,0.623868,0.235693,0.661292,0.394698
assists,0.144905,0.185739,0.419445,0.514183,0.442293,0.253284,1.000000,0.132508,0.017229,-0.030201,-0.021323,0.353955,0.092133,0.071620,-0.091370,0.455068,0.185739,0.514183,0.359490
clean_sheets,-0.123557,0.428869,0.657997,0.638013,0.509078,0.171580,0.132508,1.000000,-0.376865,-0.007302,-0.039180,-0.039180,0.133653,0.010748,0.092134,0.363096,0.428869,0.638013,0.236795
goals_conceded,-0.173736,0.506090,-0.033178,-0.056706,0.200692,0.027777,0.017229,-0.376865,1.000000,0.097197,0.111966,0.046953,0.108647,0.069024,0.118394,-0.043221,0.506090,-0.056706,-0.404887
own_goals,-0.067423,0.070681,-0.021911,-0.029062,0.005996,0.032709,-0.030201,-0.007302,0.097197,1.000000,-0.004305,-0.004305,0.029371,-0.007479,0.059103,-0.036798,0.070681,-0.029062,-0.055629


In [25]:
players_df["Full Name"] = players_df["second_name"] + ", " + players_df["first_name"] + " - " + players_df["team"]

In [26]:
players_df.to_csv('Player Data.csv')

In [27]:
#Goalies:
